In [43]:
#install all dependencies
%%bash
pip3 install gymnasium
pip3 install torch torchvision
pip3 install numpy
pip3 install matplotlib
pip3 install pandas
pip3 install scikit-learn
pip3 install tqdm
pip3 install Pillow 
pip3 install scipy
pip3 install albumentations
pip3 install torchsummary

print("Successfully Installed All Libraries")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 16.3 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 41.4 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.4/232.4 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 29.6 MB/s eta 0:00:00MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 16.7 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 978.0 kB/s eta 0:00:0031m1.8 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.7/125.7 kB 1.3 MB/s eta 0:00:00 MB/s eta 0:00:01:

In [50]:
#core PyTorch for models and tensors
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#data handling and transformations
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

#for array manipulations often used alongside PyTorch for data preprocessing
import numpy as np

#for data visualization and analysis
import matplotlib.pyplot as plt
import pandas as pd

#for splitting data, parameter search, and other utilities
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

#for building environments
import gymnasium as gym
from gymnasium import spaces

#for progress bars in loops (e.g., training loops)
from tqdm import tqdm

#for saving and loading PyTorch models
import os
import pickle

#for logging and debugging
import logging

#for working with paths and filesystems
from pathlib import Path

# Additional libraries for specific tasks or enhancements:
#for advanced neural network layers and architectures
import torch.nn.utils.prune as prune
import torchsummary

#for more complex tranformations and tensor operations not covered by PyTorch
import scipy

# For generating or working with random numbers, beyond what's available in PyTorch
import random

print('Successfully Imported All Libraries')

Successfully Imported All Libraries


In [58]:
class GameOfLifeEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, grid_size=1024, device='cpu'):
        super(GameOfLifeEnv, self).__init__()
        self.grid_size = grid_size
        self.device = torch.device(device)
        print(f'Device set to: {self.device}')
        self.observation_space = spaces.Box(low=0, high=1, shape=(grid_size, grid_size), dtype=bool)
        self.action_space = spaces.Discrete(2)
        # Initialize state
        self.state = None
        self.reset()

    def step(self, action):
        new_state = torch.zeros((self.grid_size, self.grid_size), device=self.device)
        for row in range(self.grid_size):
            for col in range(self.grid_size):
                live_neighbors = torch.sum(self.state[max(row-1, 0):min(row+2, self.grid_size), max(col-1, 0):min(col+2, self.grid_size)]) - self.state[row, col]
                # Apply Conway's rules
                if self.state[row, col] == 1 and (live_neighbors < 2 or live_neighbors > 3):
                    new_state[row, col] = 0
                elif self.state[row, col] == 0 and live_neighbors == 3:
                    new_state[row, col] = 1
                else:
                    new_state[row, col] = self.state[row, col]
        self.state = new_state
        done = False  # Define your own termination condition here
        info = {}
        return self.state.cpu().numpy(), 0, done, info

    def reset(self):
        self.state = torch.randint(0, 2, (self.grid_size, self.grid_size), device=self.device).float()
        return self.state.cpu().numpy()

    def render(self, mode='human'):
        if mode == 'human':
            print(self.state.cpu().numpy())

    def close(self):
        pass

print('done')


done


In [59]:
game = GameOfLifeEnv()
for i in range(5):
    game.step
    print(game.render('human'))

    

Device set to: cpu
[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 1. 1.]
 [0. 1. 0. ... 0. 0. 1.]
 ...
 [0. 1. 1. ... 1. 1. 0.]
 [0. 1. 1. ... 1. 0. 0.]
 [1. 0. 1. ... 0. 0. 1.]]
None
[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 1. 1.]
 [0. 1. 0. ... 0. 0. 1.]
 ...
 [0. 1. 1. ... 1. 1. 0.]
 [0. 1. 1. ... 1. 0. 0.]
 [1. 0. 1. ... 0. 0. 1.]]
None
[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 1. 1.]
 [0. 1. 0. ... 0. 0. 1.]
 ...
 [0. 1. 1. ... 1. 1. 0.]
 [0. 1. 1. ... 1. 0. 0.]
 [1. 0. 1. ... 0. 0. 1.]]
None
[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 1. 1.]
 [0. 1. 0. ... 0. 0. 1.]
 ...
 [0. 1. 1. ... 1. 1. 0.]
 [0. 1. 1. ... 1. 0. 0.]
 [1. 0. 1. ... 0. 0. 1.]]
None
[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 1. 1.]
 [0. 1. 0. ... 0. 0. 1.]
 ...
 [0. 1. 1. ... 1. 1. 0.]
 [0. 1. 1. ... 1. 0. 0.]
 [1. 0. 1. ... 0. 0. 1.]]
None
